In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, mean_absolute_error

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
import seaborn as sns 


In [2]:
df = pd.read_csv("diamonds_train.csv", index_col=0)

In [3]:
df["carat"] = 10*df["carat"]

df["cut"] = df["cut"].apply(lambda x: x.replace("Fair","0"))
df["cut"] = df["cut"].apply(lambda x: x.replace("Very Good","2"))
df["cut"] = df["cut"].apply(lambda x: x.replace("Good","1"))
df["cut"] = df["cut"].apply(lambda x: x.replace("Ideal","3"))
df["cut"] = df["cut"].apply(lambda x: x.replace("Premium","4"))
df["cut"] = df["cut"].astype(int)

df["color"] = df["color"].apply(lambda x: x.replace("D","10"))
df["color"] = df["color"].apply(lambda x: x.replace("E","9"))
df["color"] = df["color"].apply(lambda x: x.replace("F","8"))
df["color"] = df["color"].apply(lambda x: x.replace("G","6"))
df["color"] = df["color"].apply(lambda x: x.replace("H","5"))
df["color"] = df["color"].apply(lambda x: x.replace("I","4"))
df["color"] = df["color"].apply(lambda x: x.replace("J","0"))
df["color"] = df["color"].astype(int)

df["clarity"] = df["clarity"].apply(lambda x: x.replace("SI1","3"))
df["clarity"] = df["clarity"].apply(lambda x: x.replace("SI2","2"))
df["clarity"] = df["clarity"].apply(lambda x: x.replace("I1","0"))
df["clarity"] = df["clarity"].apply(lambda x: x.replace("IF","10"))
df["clarity"] = df["clarity"].apply(lambda x: x.replace("VVS1","9"))
df["clarity"] = df["clarity"].apply(lambda x: x.replace("VVS2","8"))
df["clarity"] = df["clarity"].apply(lambda x: x.replace("VS1","7"))
df["clarity"] = df["clarity"].apply(lambda x: x.replace("VS2","6"))
df["clarity"] = df["clarity"].astype(int)


In [4]:
df[df["price"] > 18000].sort_values(by="carat", ascending=False).head(20)

,carat,cut,color,clarity,depth,table,x,y,z,price
31594,35.1,4,0,6,62.5,59.0,9.66,9.63,6.03,18701
23737,30.1,1,5,2,57.6,64.0,9.44,9.38,5.42,18593
12708,30.1,0,4,2,65.8,56.0,8.99,8.94,5.90,18242
23056,30.1,4,0,2,59.7,58.0,9.41,9.32,5.59,18710
4960,28.0,1,6,2,63.8,58.0,8.90,8.85,0.00,18788
30849,26.7,1,8,2,63.8,58.0,8.69,8.64,5.54,18686
2087,26.1,3,4,2,62.1,56.0,8.85,8.73,5.46,18756
25432,26.0,3,0,2,62.5,56.0,8.81,8.76,5.49,18369
16989,25.7,4,0,3,63.0,58.0,8.77,8.65,5.49,18485
29274,25.5,4,4,7,61.8,62.0,8.70,8.65,5.36,18766


In [5]:
df[df["price"] < 800].sort_values(by="carat", ascending=False).tail(20)

,carat,cut,color,clarity,depth,table,x,y,z,price
20122,2.2,4,10,6,59.3,62.0,3.91,3.88,2.31,404
5759,2.2,0,9,6,65.1,61.0,3.87,3.78,2.49,337
5207,2.2,4,9,6,61.6,58.0,3.93,3.89,2.41,404
18590,2.2,4,8,3,60.4,61.0,3.88,3.84,2.33,342
4715,2.1,4,10,6,61.6,59.0,3.82,3.78,2.34,386
14686,2.1,4,10,6,59.1,62.0,3.89,3.86,2.29,386
27591,2.1,4,10,6,60.6,60.0,3.85,3.81,2.32,386
29393,2.1,4,9,6,60.5,59.0,3.87,3.83,2.33,386
25887,2.1,4,9,3,59.8,61.0,3.89,3.84,2.31,326
4225,2.1,4,9,6,59.6,56.0,3.93,3.89,2.33,386


In [6]:
X = np.array(df[["carat","cut", "color", "clarity"]])
y = np.array(df["price"])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 235)

In [8]:
svr_rbf = SVR(kernel='rbf', C=10000, gamma="auto", epsilon=0.1, max_iter=999999)
svr_rbf.fit(X_train, y_train)

C:\Users\xyang\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=999999).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVR(C=10000, gamma='auto', max_iter=999999)

In [9]:
y_test_pred = svr_rbf.predict(X_test)
print(r2_score(y_test, y_test_pred))
np.sqrt(mean_squared_error(y_test, y_test_pred))

0.9795639292459817


555.1367153891555

In [10]:
# C = 10000/ 555.1367153891555

In [11]:
pd.DataFrame({"y_test":y_test,"y_pred": y_test_pred})

,y_test,y_pred
0,2041,1903.567653
1,640,795.939931
2,3583,3701.451628
3,616,623.812214
4,1243,1221.798919
...,...,...
8064,11262,10915.804793
8065,1376,1582.086768
8066,1346,1349.774243
8067,11040,11811.403466


### PREDECIR TEST

In [12]:
X_pred = pd.read_csv("diamonds_test.csv", index_col = 0)

In [13]:
X_pred["carat"] = 10*X_pred["carat"]

In [ ]:
X_pred["cut"] = X_pred["cut"].apply(lambda x: x.replace("Fair","0"))
X_pred["cut"] = X_pred["cut"].apply(lambda x: x.replace("Very Good","2"))
X_pred["cut"] = X_pred["cut"].apply(lambda x: x.replace("Good","1"))
X_pred["cut"] = X_pred["cut"].apply(lambda x: x.replace("Ideal","3"))
X_pred["cut"] = X_pred["cut"].apply(lambda x: x.replace("Premium","4"))
X_pred["cut"] = X_pred["cut"].astype(int)

In [17]:


X_pred["color"] = X_pred["color"].apply(lambda x: x.replace("D","10"))
X_pred["color"] = X_pred["color"].apply(lambda x: x.replace("E","9"))
X_pred["color"] = X_pred["color"].apply(lambda x: x.replace("F","8"))
X_pred["color"] = X_pred["color"].apply(lambda x: x.replace("G","6"))
X_pred["color"] = X_pred["color"].apply(lambda x: x.replace("H","5"))
X_pred["color"] = X_pred["color"].apply(lambda x: x.replace("I","4"))
X_pred["color"] = X_pred["color"].apply(lambda x: x.replace("J","0"))
X_pred["color"] = X_pred["color"].astype(int)

X_pred["clarity"] = X_pred["clarity"].apply(lambda x: x.replace("SI1","3"))
X_pred["clarity"] = X_pred["clarity"].apply(lambda x: x.replace("SI2","2"))
X_pred["clarity"] = X_pred["clarity"].apply(lambda x: x.replace("I1","0"))
X_pred["clarity"] = X_pred["clarity"].apply(lambda x: x.replace("IF","10"))
X_pred["clarity"] = X_pred["clarity"].apply(lambda x: x.replace("VVS1","9"))
X_pred["clarity"] = X_pred["clarity"].apply(lambda x: x.replace("VVS2","8"))
X_pred["clarity"] = X_pred["clarity"].apply(lambda x: x.replace("VS1","7"))
X_pred["clarity"] = X_pred["clarity"].apply(lambda x: x.replace("VS2","6"))
X_pred["clarity"] = X_pred["clarity"].astype(int)


In [18]:
X_pred = np.array(X_pred[["carat","cut", "color", "clarity"]])

In [19]:
pd.DataFrame(X_pred)

,0,1,2,3
0,3.0,3.0,5.0,2.0
1,3.4,3.0,10.0,10.0
2,15.7,2.0,4.0,6.0
3,3.1,3.0,5.0,6.0
4,15.1,1.0,4.0,9.0
...,...,...,...,...
13444,10.5,3.0,8.0,2.0
13445,2.7,2.0,6.0,7.0
13446,15.0,4.0,10.0,6.0
13447,17.0,4.0,8.0,2.0


In [20]:
predictions_submit = svr_rbf.predict(X_pred)
predictions_submit

array([  408.14397298,  1881.3222375 ,  9701.81516169, ...,
       15287.02390032,  9423.70264847,   736.10746658])

In [15]:
pd.DataFrame(predictions_submit)

,0
0,406.116411
1,1518.506017
2,9809.310560
3,507.105117
4,9301.107762
...,...
13444,4678.042759
13445,497.939913
13446,15306.840933
13447,10139.667918


In [21]:
submission = pd.DataFrame({"id": range(len(predictions_submit)), "price": predictions_submit})

In [23]:
submission.to_csv("submission1.csv", index = False)